In [10]:
from numpy import sin, cos
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation
import methods as mt

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint


class orbits:
    
    def __init__(self,
             init_state): 
        self.init_state = np.asarray(init_state, dtype='float')
        self.time_elapsed = 0
        self.state = self.init_state
        
    def position(self):
        N=len(self.state)//5
        x = np.zeros(N)
        y = np.zeros(N)
        vx = np.zeros(N)
        vy = np.zeros(N)
        m = np.zeros(N)

        for i in range(N):
            m[i] = self.state[5*i]
            x[i] = self.state[5*i+1]
            y[i] = self.state[5*i+2]
            vx[i] = self.state[5*i+3]
            vy[i] = self.state[5*i+4]
        
        return (x, y)
        
    def generalSystem2d(self,S,t,G=6.67408e-11):
        #S0 includes m formatted mi,xi,yi,zi,vxi,vyi,vzi
        #G=6.67408e-11
        #print(S0)
        N=len(S)//5
        x = np.zeros(N)
        y = np.zeros(N)
        vx = np.zeros(N)
        vy = np.zeros(N)
        m = np.zeros(N)

        xr3 = np.zeros((N,N))
        yr3 = np.zeros((N,N))

        dvx = np.zeros(N)
        dvy = np.zeros(N)

        dm = np.zeros(N)

        for i in range(N):
            m[i] = S[5*i]
            x[i] = S[5*i+1]
            y[i] = S[5*i+2]
            vx[i] = S[5*i+3]
            vy[i] = S[5*i+4]


        for i in range(N):
            for j in range(N):
                if(i!=j):
                    r = np.sqrt((x[i]-x[j])**2+(y[i]-y[j])**2)
                    xr3[i][j] = (x[i]-x[j])/(r**3)
                    yr3[i][j] = (y[i]-y[j])/(r**3)


        for i in range(N):
            for j in range(N):
                if(i!=j):
                    dvx[i] = dvx[i]-G*m[j]*xr3[i][j]
                    dvy[i] = dvy[i]-G*m[j]*yr3[i][j]


        return_array = np.array([])

        for i in range(N):
            return_array = np.append(return_array,[dm[i],vx[i],vy[i],dvx[i],dvy[i]])

        return return_array

    def step(self, dt):
        self.state = integrate.odeint(self.generalSystem2d, self.state, [0, dt])[1]
        self.time_elapsed += dt
    

G=6.67408e-11
d=1
me=5.97219e24
xe0=149.6e9
ye0=0
ze0=0
vye0=29814
ms=1.989e30
S0=np.array([ms,0,0,0,0,0,0,me,xe0,ye0,ze0,0,vye0,0])
orbit = orbits(S0)
dt = 50
fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal',autoscale_on=False,
                     xlim=(-2e11, 2e11), ylim=(-2e11, 2e11))
ax.grid()
line, = ax.plot([],[], 'o-', lw=2)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

def init():
    """initialize animation"""
    line.set_data([],[])
    time_text.set_text('')
    return line, time_text

def animate(i):
    """perform animation step"""
    global orbit, dt
    orbit.step(dt)
    
    line.set_data(*orbit.position())
    time_text.set_text('time = %.1f' % orbit.time_elapsed)
    return line, time_text, 

# choose the interval based on dt and the time to animate one step
from time import time
t0 = time()
animate(0)
t1 = time()
interval = 1000 * dt - (t1 - t0)

ani = animation.FuncAnimation(fig, animate, frames=300,
                              interval=interval, blit=True, init_func=init)

# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
#ani.save('double_pendulum.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

plt.show()